In [1]:
# -*- coding: utf-8 -*-
"""
Sophisticated Voice Assistant - Jupyter Notebook Version

This notebook guides you through creating a voice assistant capable of:
- Checking the weather
- Playing music (via YouTube search)
- Setting basic reminders (console output)
- Managing Google Calendar events

**Setup Required BEFORE Running Cells:**

1.  **Install Libraries:** Run the next cell (`!pip install ...`).
2.  **API Keys:**
    *   **OpenWeatherMap:** Get a free API key from https://openweathermap.org/appid
    *   **Google Calendar:** Follow the prerequisite steps to enable the API and download `credentials.json`.
3.  **Create `.env` file:**
    *   In the SAME DIRECTORY as this notebook, create a text file named `.env` (literally, starting with a dot).
    *   Add the following lines to it, replacing the placeholder values with YOUR actual information:

      ```dotenv
      # .env file
      OPENWEATHERMAP_API_KEY=YOUR_OPENWEATHERMAP_API_KEY_HERE
      DEFAULT_CITY=YourCityName # e.g., London
      YOUR_TIMEZONE=Your/Timezone # e.g., Europe/London or America/New_York
      ```
      *Find your timezone string from the "TZ database name" column here: https://en.wikipedia.org/wiki/List_of_tz_database_time_zones*
4.  **Place `credentials.json`:** Make sure the `credentials.json` file you downloaded from Google Cloud Console is in the SAME DIRECTORY as this notebook.
5.  **Microphone:** Ensure your microphone is properly connected and configured in your OS.
"""
print("Setup instructions complete. Proceed to the next cell to install libraries.")

Setup instructions complete. Proceed to the next cell to install libraries.


In [2]:
!pip install SpeechRecognition PyAudio pyttsx3 requests python-dotenv pytz google-api-python-client google-auth-oauthlib google-auth-httplib2

In [3]:
import speech_recognition as sr
import pyttsx3
import datetime
import pytz # For timezone handling
import requests
import json
import webbrowser
import os
import time
from dotenv import load_dotenv # For loading environment variables
import requests
# --- Google Calendar Imports ---
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

print("Libraries imported successfully.")

Libraries imported successfully.


In [4]:
# Load environment variables from .env file
load_dotenv()

# --- Configuration (Loaded from .env) --- ## --- REPLACE VALUES IN .env FILE --- ##
WEATHER_API_KEY = os.getenv('OPENWEATHERMAP_API_KEY')
DEFAULT_CITY = os.getenv('DEFAULT_CITY', 'Navi Mumbai') # Default city if not set in .env
YOUR_TIMEZONE = os.getenv('YOUR_TIMEZONE', 'Etc/UTC') # Default to UTC if not set

# --- Google Calendar Setup ---
SCOPES = ['https://www.googleapis.com/auth/calendar']
CREDENTIALS_FILE = 'credentials.json' # Should be in the same directory
TOKEN_FILE = 'token.json' # Will be created automatically after first auth

# --- Basic Validation ---
if not WEATHER_API_KEY:
    print("⚠️ WARNING: OPENWEATHERMAP_API_KEY not found in .env file. Weather functionality will fail.")
if not os.path.exists(CREDENTIALS_FILE):
    print(f"⚠️ WARNING: Google Calendar credentials file ('{CREDENTIALS_FILE}') not found. Calendar functions will fail.")
else:
    print(f"Found '{CREDENTIALS_FILE}'.")

print(f"Default city set to: {DEFAULT_CITY}")
print(f"Timezone set to: {YOUR_TIMEZONE}")

Found 'credentials.json'.
Default city set to: Navi Mumbai
Timezone set to: Asia/Kolkata


In [5]:
# --- Text-to-Speech Engine Setup ---
try:
    engine = pyttsx3.init()
    voices = engine.getProperty('voices')
    # You can experiment with different voices if available
    # engine.setProperty('voice', voices[1].id) # Index 0 is often male, 1 female
    engine.setProperty('rate', 180) # Adjust speech rate (words per minute)
    print("TTS Engine Initialized.")
except Exception as e:
    print(f"❌ Error initializing TTS engine: {e}")
    engine = None # Set engine to None if initialization fails

# --- Speech Recognition Setup ---
recognizer = sr.Recognizer()
microphone = sr.Microphone()

# Adjust recognizer settings for better performance
recognizer.pause_threshold = 0.8 # Seconds of non-speaking audio before phrase is considered complete
recognizer.energy_threshold = 400 # Minimum audio energy to consider for recording (adjust based on mic sensitivity and noise)
# recognizer.dynamic_energy_threshold = True # Can help in varying noise levels

print("Speech Recognizer Initialized.")

TTS Engine Initialized.
Speech Recognizer Initialized.


In [6]:
def speak(text):
    """Converts text to speech."""
    print(f"Assistant: {text}")
    if engine:
        try:
            engine.say(text)
            engine.runAndWait()
        except Exception as e:
            print(f"❌ TTS Error: {e}")
    else:
        print("(TTS Engine not available)")

def listen():
    """Listens for command via microphone and returns the recognized text."""
    with microphone as source:
        print("\n👂 Listening...")
        # Adjust for ambient noise dynamically or once initially
        recognizer.adjust_for_ambient_noise(source, duration=0.5)
        try:
            # Listen with timeouts to avoid waiting indefinitely
            audio = recognizer.listen(source, timeout=5, phrase_time_limit=10)
            print("🧠 Recognizing...")
            # Use Google Speech Recognition (requires internet)
            command = recognizer.recognize_google(audio).lower()
            print(f"🗣️ You said: {command}")
            return command
        except sr.WaitTimeoutError:
            print("묵 No speech detected within timeout.")
            return None # Return None if no speech is detected
        except sr.UnknownValueError:
            speak("Sorry, I didn't quite catch that. Could you please repeat?")
            print("❓ Could not understand audio.")
            return None
        except sr.RequestError as e:
            speak(f"Sorry, could not request results from the speech recognition service; {e}")
            print(f"❌ API Request Error: {e}")
            return None
        except Exception as e:
            speak("An unexpected error occurred during listening.")
            print(f"❌ Listening Error: {e}")
            return None

# --- Test the functions ---
# speak("Testing the speak function. Can you hear me?")
# time.sleep(0.5)
# speak("Now, I will try to listen. Say something after 'Listening...' appears.")
# test_command = listen()
# if test_command:
#     speak(f"I heard you say: {test_command}")
# else:
#     speak("I didn't hear anything clearly during the test.")

In [7]:
def get_weather(city):
    """Fetches and speaks the weather for a given city using OpenWeatherMap."""
    if not WEATHER_API_KEY:
        speak("Weather API key is not configured. Cannot fetch weather.")
        return

    base_url = "http://api.openweathermap.org/data/2.5/weather?"
    complete_url = f"{base_url}appid={WEATHER_API_KEY}&q={city}&units=metric" # Using metric units (Celsius)

    try:
        response = requests.get(complete_url)
        response.raise_for_status() # Raise an HTTPError for bad responses (4xx or 5xx)
        weather_data = response.json()

        if weather_data.get("cod") != 200 and weather_data.get("cod") != "200":
             # Handle cases where the API returns an error code within the JSON
             error_message = weather_data.get("message", "Unknown error")
             speak(f"Sorry, I couldn't find weather data for {city}. Reason: {error_message}")
             print(f"Weather API Error for {city}: {weather_data}")
             return

        main = weather_data.get("main", {})
        weather_desc_list = weather_data.get("weather", [{}])
        weather_desc = weather_desc_list[0].get("description", "No description available") if weather_desc_list else "No description available"
        temp = main.get("temp")
        feels_like = main.get("feels_like")
        humidity = main.get("humidity")
        wind_data = weather_data.get("wind", {})
        wind_speed = wind_data.get("speed") # Speed in meter/sec for metric units

        if temp is None:
             speak(f"Sorry, I couldn't get the temperature details for {city}.")
             return

        report = (f"The weather in {city.capitalize()} is currently {weather_desc}. "
                  f"The temperature is {temp:.1f} degrees Celsius")
        if feels_like is not None:
             report += f", feeling like {feels_like:.1f} degrees."
        else:
             report += "." # End sentence if feels_like is missing

        if humidity is not None:
            report += f" Humidity is at {humidity} percent."
        if wind_speed is not None:
             report += f" Wind speed is {wind_speed:.1f} meters per second."

        speak(report)

    except requests.exceptions.HTTPError as http_err:
         if response.status_code == 401:
              speak("Authentication failed for weather service. Please check your API key.")
         elif response.status_code == 404:
              speak(f"Sorry, I couldn't find the city: {city}.")
         else:
              speak(f"An HTTP error occurred while fetching weather: {http_err}")
         print(f"HTTP Error fetching weather for {city}: {http_err}")
    except requests.exceptions.RequestException as req_err:
        speak(f"Sorry, I couldn't connect to the weather service. Error: {req_err}")
        print(f"Connection Error fetching weather: {req_err}")
    except KeyError as key_err:
        speak(f"Error parsing weather data for {city}. Unexpected data format. Missing key: {key_err}")
        print(f"KeyError parsing weather data: {key_err} - Data: {weather_data}")
    except Exception as e:
        speak(f"An unexpected error occurred while fetching weather for {city}.")
        print(f"❌ Unexpected Weather Error: {e}")

# --- Test the weather function ---
# speak(f"Let's test the weather function for the default city: {DEFAULT_CITY}")
# get_weather(DEFAULT_CITY)
# time.sleep(1)
# speak("Now testing for a specific city, say, Paris.")
# get_weather("Paris")
# time.sleep(1)
# speak("Testing for a non-existent city.")
# get_weather("CityThatDoesNotExist123")

In [8]:
def play_music(query):
    """Opens YouTube search results for the music query in a web browser."""
    if not query:
        speak("Please specify what music you want to play.")
        return
    speak(f"Okay, searching for '{query}' on YouTube.")
    try:
        search_url = f"https://www.youtube.com/results?search_query={query.replace(' ', '+')}"
        webbrowser.open(search_url)
        speak("I've opened the search results in your web browser.")
    except Exception as e:
        speak("Sorry, I couldn't open the web browser.")
        print(f"❌ Error opening web browser: {e}")

# --- Test the music function ---
# speak("Testing music function. Let's search for 'classical piano mix'.")
# play_music("classical piano mix")

In [9]:
import json # Make sure json is imported
import os   # Make sure os is imported

REMINDER_FILE = 'reminders.json'

def load_reminders():
    """Loads reminders from the JSON file."""
    global reminders # Need global to modify the list defined outside
    if os.path.exists(REMINDER_FILE):
        try:
            with open(REMINDER_FILE, 'r') as f:
                reminders = json.load(f)
            print(f"INFO: Loaded {len(reminders)} reminders from {REMINDER_FILE}")
        except json.JSONDecodeError:
            print(f"ERROR: Could not decode JSON from {REMINDER_FILE}. Starting with empty list.")
            reminders = []
        except Exception as e:
            print(f"ERROR: Error loading reminders: {e}")
            reminders = [] # Reset if file is corrupt
    else:
        print(f"INFO: Reminder file {REMINDER_FILE} not found. Starting with empty list.")
        reminders = [] # Ensure reminders is an empty list if file doesn't exist

def save_reminders():
    """Saves the current reminders list to the JSON file."""
    global reminders
    try:
        with open(REMINDER_FILE, 'w') as f:
            json.dump(reminders, f, indent=4) # Use indent for readability
        # print(f"INFO: Saved {len(reminders)} reminders to {REMINDER_FILE}") # Optional: print on save
    except Exception as e:
        print(f"ERROR: Error saving reminders: {e}")
# --- Basic Reminder List ---
reminders = [] # Simple list to hold reminders in memory

def set_reminder(reminder_text):
    """Sets a simple reminder (adds to a list and confirms)."""
    if not reminder_text:
        speak("What should I remind you about?")
        return

    now = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    reminders.append({"text": reminder_text, "set_at": now})
    speak(f"Okay, I will remember: '{reminder_text}'.")
    print(f"--- Reminder Added ---")
    print(f"Task: {reminder_text}")
    print(f"Set at: {now}")
    print(f"Current Reminders: {len(reminders)}")
    print(f"----------------------")
    save_reminders()
    # In a real application, you'd schedule this using `threading.Timer`,
    # `schedule` library, or integrate with a calendar/task service.
    # You would also need proper date/time parsing from the user command.

def show_reminders():
    """Speaks out the currently stored reminders."""
    if not reminders:
        speak("You have no reminders set right now.")
        return

    speak(f"Okay, you have {len(reminders)} reminder{'s' if len(reminders) > 1 else ''}:")
    for i, reminder in enumerate(reminders):
        speak(f"Reminder {i+1}: {reminder['text']} (set at {reminder['set_at']})")
        time.sleep(0.5) # Pause slightly between reminders

# --- Test reminder functions ---
# speak("Testing reminders. Let's add one.")
# set_reminder("Call Mom later today")
# time.sleep(1)
# set_reminder("Buy groceries")
# time.sleep(1)
# speak("Now let's check the reminders.")
# show_reminders()

In [10]:
def get_google_calendar_service():
    """Authenticates and returns the Google Calendar API service client."""
    creds = None
    # The file token.json stores the user's access and refresh tokens.
    if os.path.exists(TOKEN_FILE):
        try:
            creds = Credentials.from_authorized_user_file(TOKEN_FILE, SCOPES)
        except Exception as e:
            print(f"Error loading token file '{TOKEN_FILE}': {e}. Will try to re-authenticate.")
            creds = None # Force re-authentication

    # If there are no (valid) credentials available, let the user log in.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            try:
                print("Refreshing calendar access token...")
                creds.refresh(Request())
                print("Token refreshed.")
            except Exception as e:
                speak(f"Could not refresh calendar token: {e}. You might need to re-authenticate.")
                print(f"❌ Token Refresh Error: {e}")
                # Optionally remove the token file to force full re-auth
                try:
                    os.remove(TOKEN_FILE)
                    print(f"Removed invalid token file '{TOKEN_FILE}'. Please run the command again to re-authenticate.")
                except OSError as oe:
                     print(f"Could not remove token file '{TOKEN_FILE}': {oe}")
                return None # Stop if refresh fails
        else:
            # This is where the browser window will open for the FIRST time
            if not os.path.exists(CREDENTIALS_FILE):
                speak(f"Calendar credentials file ('{CREDENTIALS_FILE}') not found. Cannot authenticate.")
                print(f"❌ Missing {CREDENTIALS_FILE}")
                return None
            try:
                print(f"Starting authentication flow using '{CREDENTIALS_FILE}'...")
                flow = InstalledAppFlow.from_client_secrets_file(CREDENTIALS_FILE, SCOPES)
                # run_local_server will open the browser for authentication
                creds = flow.run_local_server(port=0) # Use port=0 to find a free port
                print("Authentication successful.")
            except FileNotFoundError:
                 speak(f"Credentials file '{CREDENTIALS_FILE}' not found. Please place it in the notebook directory.")
                 print(f"❌ FileNotFoundError: {CREDENTIALS_FILE}")
                 return None
            except Exception as e:
                 speak(f"Calendar authentication failed: {e}. Check console for details.")
                 print(f"❌ Authentication Error: {e}")
                 return None

        # Save the credentials for the next run
        try:
            with open(TOKEN_FILE, 'w') as token:
                token.write(creds.to_json())
            print(f"Credentials saved to '{TOKEN_FILE}'.")
        except IOError as e:
            speak(f"Could not save calendar token file: {e}")
            print(f"❌ Error saving token: {e}")

    # Build and return the service object
    try:
        service = build('calendar', 'v3', credentials=creds)
        print("Google Calendar service client created successfully.")
        return service
    except HttpError as error:
        speak(f"An error occurred building the Calendar service: {error}")
        print(f"❌ Calendar Service Build HttpError: {error}")
        return None
    except Exception as e:
        speak(f"Failed to build Google Calendar service: {e}")
        print(f"❌ Calendar Service Build Error: {e}")
        return None


def add_calendar_event(summary, description, start_time_str, end_time_str):
    """Adds an event to the primary Google Calendar."""
    # !! VERY IMPORTANT !!
    # This function currently expects ISO format strings with timezone offset
    # E.g., '2024-08-15T10:00:00+01:00' for BST or
    # '2024-08-15T09:00:00-04:00' for EDT
    # Natural Language Date/Time parsing ("tomorrow at 3pm") is NOT implemented here.
    # We will use pytz to construct the correct format from a simpler input for demo purposes.

    service = get_google_calendar_service()
    if not service:
        speak("Cannot access Google Calendar service. Please check setup and authentication.")
        return

    try:
        # Attempt to parse the input strings and add timezone info
        # Example assumes input like '2024-08-15 10:00'
        # A more robust solution needs proper NLP date/time extraction.
        local_tz = pytz.timezone(YOUR_TIMEZONE) ## --- USES TIMEZONE FROM .env --- ##

        # Naive parsing - assumes YYYY-MM-DD HH:MM format
        naive_start_dt = datetime.datetime.strptime(start_time_str, '%Y-%m-%d %H:%M')
        naive_end_dt = datetime.datetime.strptime(end_time_str, '%Y-%m-%d %H:%M')

        # Make the datetime objects timezone-aware
        aware_start_dt = local_tz.localize(naive_start_dt)
        aware_end_dt = local_tz.localize(naive_end_dt)

        # Format for Google Calendar API (RFC3339)
        start_iso = aware_start_dt.isoformat()
        end_iso = aware_end_dt.isoformat()

        event = {
            'summary': summary,
            'description': description,
            'start': {
                'dateTime': start_iso,
                # 'timeZone': str(local_tz), # Timezone included in dateTime offset
            },
            'end': {
                'dateTime': end_iso,
                # 'timeZone': str(local_tz),
            },
            # Optional: Add reminders (popup, email)
            'reminders': {
                'useDefault': False,
                'overrides': [
                    {'method': 'popup', 'minutes': 30}, # 30 min popup before event
                 ],
             },
        }

        speak(f"Okay, adding '{summary}' to your Google Calendar from {start_time_str} to {end_time_str}.")
        print(f"Creating event: {event}")

        created_event = service.events().insert(calendarId='primary', body=event).execute()

        event_link = created_event.get('htmlLink', 'No link available')
        speak(f"Event created successfully!")
        print(f"Event created: {event_link}")
        try:
            webbrowser.open(event_link) # Try opening the event link
        except Exception:
             speak("You can view it in your Google Calendar.")


    except ValueError as ve:
        speak("Sorry, I couldn't understand the date or time format. Please use YYYY-MM-DD HH:MM format for now.")
        print(f"❌ Date/Time parsing error: {ve}. Input: Start='{start_time_str}', End='{end_time_str}'")
    except HttpError as error:
        speak(f"An error occurred while creating the calendar event: {error}")
        print(f"❌ Calendar Event Creation HttpError: {error}")
    except pytz.UnknownTimeZoneError:
         speak(f"The timezone '{YOUR_TIMEZONE}' set in the .env file is invalid. Please check the timezone list.")
         print(f"❌ Invalid Timezone: {YOUR_TIMEZONE}")
    except Exception as e:
        speak(f"An unexpected error occurred while creating the calendar event.")
        print(f"❌ Unexpected Calendar Error: {e}")


# --- Test Calendar Functions ---
# speak("Testing Google Calendar. First, let's ensure authentication works.")
# test_service = get_google_calendar_service()
# if test_service:
#     speak("Authentication successful or token loaded.")
#     speak("Now, let's try adding a test event for '2024-12-25 10:00' to '2024-12-25 11:00'.")
#     add_calendar_event(
#         summary="Test Event from Voice Assistant",
#         description="This is a test event added via the Python script.",
#         start_time_str="2024-12-25 10:00", # Use a future date/time for testing
#         end_time_str="2024-12-25 11:00"
#     )
# else:
#     speak("Could not get Calendar service. Check previous errors.")
# Note: The first time you run this involving `get_google_calendar_service`,
# it should open a web browser asking you to log in to Google and grant permissions.

In [13]:
# CELL 11 Code - Fully Replaced

import re # Ensure re is imported if you use regex later
import datetime # Ensure datetime is imported
import pytz # Ensure pytz is imported

def process_command(command):
    """Parses the command and executes the corresponding action."""
    if not command:
        return True # Continue listening if listen() returned None

    processed = False # Flag to check if any command was matched

    # --- Basic Commands ---
    if "hello" in command or "hi" in command or "hey assistant" in command:
        speak("Hello there! How can I help you today?")
        processed = True

    elif "what time is it" in command or "current time" in command:
        # Make sure YOUR_TIMEZONE is defined (should be loaded from .env in Cell 4)
        try:
            now = datetime.datetime.now(pytz.timezone(YOUR_TIMEZONE))
            speak(f"The current time in {YOUR_TIMEZONE} is {now.strftime('%I:%M %p')}.") # e.g., 03:45 PM
        except NameError:
             speak("Timezone information is missing. Cannot get current time.")
             print("ERROR: YOUR_TIMEZONE variable not defined.")
        except Exception as e:
             speak("Sorry, I couldn't get the current time.")
             print(f"Error getting time: {e}")
        processed = True

    elif "what is today's date" in command or "date today" in command:
         today = datetime.date.today()
         speak(f"Today's date is {today.strftime('%B %d, %Y')}.") # e.g., July 26, 2023
         processed = True

    # --- Weather ---
    # Use 'elif' to avoid multiple triggers if keywords overlap
    elif "weather in" in command or "forecast for" in command:
        # Extract city name after the trigger phrase
        city = None
        try:
            if "weather in" in command:
                city = command.split("weather in")[-1].strip()
            elif "forecast for" in command:
                city = command.split("forecast for")[-1].strip()

            if city:
                get_weather(city)
            else:
                # Fallback if extraction failed but keyword was present
                speak("Which city's weather would you like?")
        except Exception as e:
            speak("Sorry, I had trouble understanding the city name.")
            print(f"Error parsing city from command '{command}': {e}")
        processed = True
    elif "weather" in command or "forecast" in command:
         # Important: This elif should come AFTER the specific city check
         # Only run if the more specific check didn't already handle it
         if not processed:
             # Make sure DEFAULT_CITY is defined (should be loaded from .env in Cell 4)
             try:
                  speak(f"Getting the weather for your default city: {DEFAULT_CITY}")
                  get_weather(DEFAULT_CITY)
             except NameError:
                  speak("Default city information is missing. Cannot get weather.")
                  print("ERROR: DEFAULT_CITY variable not defined.")
             except Exception as e:
                  speak("Sorry, I encountered an error getting the default weather.")
                  print(f"Error getting default weather: {e}")
             processed = True # Mark as processed here

    # --- Music ---
    elif "play music" in command or "play song" in command or "search youtube for" in command:
        query = "" # Initialize query
        processed = True # Assume processed unless error happens before end

        try:
            # Determine the trigger phrase and extract text AFTER it
            trigger_phrase = None
            if "search youtube for" in command: # Check longest phrase first
                 trigger_phrase = "search youtube for"
            elif "play music" in command:
                 trigger_phrase = "play music"
            elif "play song" in command:
                 trigger_phrase = "play song"

            if trigger_phrase:
                # Find the last occurrence of the trigger and get text after it
                start_index = command.rfind(trigger_phrase) + len(trigger_phrase)
                query = command[start_index:].strip()

            # Now check if we have a valid query
            if query:
                play_music(query) # Assumes play_music function exists
            else:
                # If a trigger was found but query is empty (e.g., user just said "play music")
                speak("What music or song would you like me to search for?")
                # In a more advanced version, you'd wait for the next command here specifically

        except NameError:
             speak("The music playing function is not available.")
             print("ERROR: play_music function not defined.")
             processed = True # Still processed, even if function missing
        except Exception as e:
            speak("Sorry, I had trouble processing the music request.")
            print(f"Error processing music command '{command}': {e}")
            processed = True # Ensure processed is true even on error


    # --- Reminders (Basic) ---
    elif "remind me to" in command:
        try:
            reminder_text = command.split("remind me to")[-1].strip()
            if reminder_text:
                set_reminder(reminder_text) # Assumes set_reminder function exists
            else:
                speak("What should I remind you about?")
        except NameError:
             speak("The reminder function is not available.")
             print("ERROR: set_reminder function not defined.")
        except Exception as e:
            speak("Sorry, I had trouble setting that reminder.")
            print(f"Error parsing reminder from command '{command}': {e}")
        processed = True
    elif "show reminders" in command or "what are my reminders" in command:
         try:
            show_reminders() # Assumes show_reminders function exists
         except NameError:
            speak("The reminder viewing function is not available.")
            print("ERROR: show_reminders function not defined.")
         processed = True

    # --- Calendar (Basic - requires specific spoken format for now) ---
    elif "add calendar event" in command or "schedule event" in command:
         # This part needs significant improvement with NLP for natural language dates/times
         speak("Okay, I can try to add an event. I'll need the details in a specific format.")
         speak("First, what is the event summary or title?")
         summary = listen()
         if not summary: return True # Exit if nothing heard

         speak("Next, the start date and time? Please say it like 'Year Month Day Hour Minute', for example '2024 August 15 10 00'.")
         start_input = listen()
         if not start_input: return True

         speak("Finally, the end date and time? Use the same format, 'Year Month Day Hour Minute'.")
         end_input = listen()
         if not end_input: return True

         speak("Any description for the event? Say 'no description' if not.")
         description = listen()
         desc_text = ""
         if description and "no description" not in description:
            desc_text = description

         # --- Attempt to parse the structured input ---
         try: # <<< Indentation Level 1 (inside elif)
             start_parts = start_input.split()
             end_parts = end_input.split()
             if len(start_parts) == 5 and len(end_parts) == 5: # <<< Indentation Level 2
                 # Convert month name to number (basic handling)
                 # Ensure month names are generated correctly for reliable mapping
                 month_map = {datetime.date(2000, i, 1).strftime('%B').lower(): str(i).zfill(2) for i in range(1, 13)}
                 month_map.update({datetime.date(2000, i, 1).strftime('%b').lower(): str(i).zfill(2) for i in range(1, 13)})

                 start_month_num = month_map.get(start_parts[1].lower())
                 end_month_num = month_map.get(end_parts[1].lower())

                 if start_month_num and end_month_num: # <<< Indentation Level 3
                     # Construct date/time strings in the required YYYY-MM-DD HH:MM format
                     start_str = f"{start_parts[0]}-{start_month_num}-{start_parts[2].zfill(2)} {start_parts[3].zfill(2)}:{start_parts[4].zfill(2)}"
                     end_str = f"{end_parts[0]}-{end_month_num}-{end_parts[2].zfill(2)} {end_parts[3].zfill(2)}:{end_parts[4].zfill(2)}"
                     print(f"Attempting to parse start='{start_str}', end='{end_str}'")

                     # --- Confirmation Start ---
                     speak(f"Okay, I have the event: '{summary}' starting {start_str} and ending {end_str}. Should I add this to your calendar? Say yes or no.")
                     confirmation = listen()
                     if confirmation and ("yes" in confirmation or "confirm" in confirmation or "do it" in confirmation or "yeah" in confirmation): # <<< Indentation Level 4
                         speak("Okay, adding the event.")
                         add_calendar_event(summary, desc_text, start_str, end_str) # Assumes add_calendar_event exists
                     else: # <<< Indentation Level 4
                         speak("Okay, I won't add the event.")
                     # --- Confirmation End ---

                 else: # <<< Indentation Level 3 (Aligned with 'if start_month_num...')
                      speak("Sorry, I couldn't understand the month name.")
             else: # <<< Indentation Level 2 (Aligned with 'if len(start_parts)...')
                 speak("Sorry, the date/time wasn't in the expected 'Year Month Day Hour Minute' format.")

         except NameError as ne: # Specific check if add_calendar_event is missing
              speak("The calendar event function is not available.")
              print(f"ERROR: Function needed for calendar not defined: {ne}")
         except Exception as e: # <<< CORRECT INDENTATION (Aligned with 'try:') - Level 1
             speak("Sorry, I encountered an error processing the calendar event details.") # <<< CORRECT INDENTATION - Level 2
             print(f"Error parsing/adding calendar event from input: {e}") # <<< CORRECT INDENTATION - Level 2

         processed = True # <<< CORRECT INDENTATION (Aligned with 'try:' and 'except:') - Level 1

    # --- Jokes --- (Add this block back if you implemented it)
    elif "tell me a joke" in command or "make me laugh" in command or "tell a joke" in command:
         speak("Okay, let me find a good one...")
         try:
             # Using icanhazdadjoke API
             headers = {'Accept': 'application/json'}
             response = requests.get("https://icanhazdadjoke.com/", headers=headers)
             response.raise_for_status() # Raise an exception for bad status codes
             joke_data = response.json()
             joke = joke_data.get("joke")
             if joke:
                  speak(joke)
                  time.sleep(0.5) # Pause slightly after telling joke
                  speak("Hope you liked it!")
             else:
                  speak("Sorry, I couldn't get the joke text, even though the request seemed okay.")
         except requests.exceptions.RequestException as e:
             speak("Sorry, I couldn't connect to the joke service right now.")
             print(f"Joke API error: {e}")
         except json.JSONDecodeError: # Ensure json is imported
              speak("Sorry, the joke service gave a response I couldn't understand.")
              print(f"Joke API JSON decode error.")
         except Exception as e:
              speak("Sorry, an unexpected error occurred while getting a joke.")
              print(f"Unexpected Joke error: {e}")
         processed = True

    # --- Exit --- (Correct position after other commands)
    elif "goodbye" in command or "exit" in command or "stop listening" in command or "shut down" in command:
        speak("Goodbye! Have a great day.")
        return False # Signal to exit the main loop

    # --- Fallback ---
    if not processed and command: # Only react if command was heard but not processed
        # Avoid responding to every unrecognized sound
        # speak(f"Sorry, I didn't understand the command: {command}")
        print(f"Command not recognized: '{command}'")

    return True # Continue listening by default

# You might need to define the functions called here (speak, listen, get_weather, etc.)
# Ensure necessary imports like datetime, pytz, requests, json, time are present at the top of the file or relevant cells.
# Ensure variables like YOUR_TIMEZONE, DEFAULT_CITY are defined and loaded.

# print("Command processing function defined.") # Optional: Keep this line outside the function definition

In [ ]:
# CELL 12 Code - Fully Replaced

# --- Main Execution Loop ---
if __name__ == "__main__": # Check if running as main script/cell
    # --- Initialization ---
    reminders = [] # Initialize empty first (good practice)
    load_reminders() # Load saved reminders if implemented and defined
    speak("Initializing Assistant...")

    # --- Optional: Greet based on time ---
    try: # <<< CORRECT INDENTATION (Level 1 - inside 'if')
        # Ensure YOUR_TIMEZONE is loaded from .env (Cell 4) and pytz/datetime imported (Cell 3)
        local_tz = pytz.timezone(YOUR_TIMEZONE)
        hour = datetime.datetime.now(local_tz).hour
        if 5 <= hour < 12:
            speak("Good morning!")
        elif 12 <= hour < 18:
            speak("Good afternoon!")
        else:
            speak("Good evening!")
    except NameError:
        # Handle case where YOUR_TIMEZONE isn't defined
        print("Warning: YOUR_TIMEZONE not defined. Using default greeting.")
        speak("Hello!")
    except Exception as e:
         print(f"Warning: Could not determine time for greeting: {e}") # Non-critical if timezone fails here
         speak("Hello!") # Default greeting on error

    # --- Ready Prompt ---
    speak("How can I help you?") # <<< CORRECT INDENTATION (Level 1 - inside 'if')

    # --- Main Loop Start ---
    running = True # <<< CORRECT INDENTATION (Level 1 - inside 'if')
    while running: # <<< CORRECT INDENTATION (Level 1 - inside 'if')
        try: # <<< CORRECT INDENTATION (Level 2 - inside 'while')
            command = listen() # Assumes listen() is defined
            if command is not None: # Only process if something was heard and recognized
                running = process_command(command) # Assumes process_command() is defined
            # Add a small delay to prevent tight looping if listen keeps failing silently
            # This also makes the CPU usage slightly lower.
            time.sleep(0.1) # Assumes time is imported

        except KeyboardInterrupt: # <<< CORRECT INDENTATION (Level 2 - inside 'while', aligned with 'try')
             print("\nKeyboardInterrupt detected. Shutting down...")
             speak("Shutting down now. Goodbye!")
             running = False
        except Exception as e: # <<< CORRECT INDENTATION (Level 2 - inside 'while', aligned with 'try')
             print(f"\n--- UNEXPECTED ERROR IN MAIN LOOP ---")
             print(f"Error: {e}")
             # Log the full traceback for debugging if needed:
             # import traceback
             # traceback.print_exc()
             speak("An unexpected error occurred in the main loop. Please check the console. Trying to continue.")
             # Avoid immediate crash, try to continue
             time.sleep(2)

    # --- Shutdown Message ---
    print("Assistant has stopped.") # <<< CORRECT INDENTATION (Level 1 - inside 'if', after 'while')

else: # <<< CORRECT INDENTATION (Level 0 - aligned with 'if __name__...')
     print("Main loop execution skipped (likely because cell was not run directly or environment issues).")


# --- How to Stop ---
# To stop the loop while it's running in Jupyter, you need to interrupt the kernel.
# Go to the "Kernel" menu above and select "Interrupt".
# You might need to press it multiple times if it's stuck in a blocking call.

INFO: Reminder file reminders.json not found. Starting with empty list.
Assistant: Initializing Assistant...
Assistant: Good afternoon!
Assistant: How can I help you?

👂 Listening...
🧠 Recognizing...
🗣️ You said: hello
Assistant: Hello there! How can I help you today?

👂 Listening...
🧠 Recognizing...
🗣️ You said: search youtube for song
Assistant: Okay, searching for 'song' on YouTube.
Assistant: I've opened the search results in your web browser.

👂 Listening...
🧠 Recognizing...
Assistant: Sorry, I didn't quite catch that. Could you please repeat?
❓ Could not understand audio.

👂 Listening...
🧠 Recognizing...
🗣️ You said: what's the weather today
Assistant: Getting the weather for your default city: Navi Mumbai
Assistant: The weather in Navi mumbai is currently clear sky. The temperature is 30.5 degrees Celsius, feeling like 34.1 degrees. Humidity is at 61 percent. Wind speed is 6.1 meters per second.

👂 Listening...
🧠 Recognizing...
🗣️ You said: any reminder for me
Command not recogn